In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *
import pymysql

In [2]:
connection = pymysql.connect(
        host='localhost',
        user='root',
        password='root',
        database='main_database'
    )

In [3]:
spark = SparkSession.builder.appName("hadoop_add_file").config("spark.jars", "C:\spark-3.5.1-bin-hadoop3\jars\mysql-connector-j-8.4.0.jar").getOrCreate()

In [4]:
url = "jdbc:mysql://localhost:3306/main_database"
properties = {
    "user": "root",
    "password": "root",
    "driver": "com.mysql.jdbc.Driver"
}

In [5]:
df = spark.read.jdbc(url=url, table='cf_etl_table', properties=properties)
df.show()
is_inc = df.select('is_incremental').collect()[0]['is_incremental']
is_inc

+---+--------------+-----------+--------------------+--------------+-------------------+-------------------+--------------+-------------------+---------+
| id|  Schema_names|Table_names|            location|hdfs_file_name|    start_date_time|      end_date_time|is_incremental|     execution_date|inc_field|
+---+--------------+-----------+--------------------+--------------+-------------------+-------------------+--------------+-------------------+---------+
|  1|test_database1|sample_data|hdfs://localhost:...|   sample_data|2023-06-28 00:00:01|2023-06-28 23:59:59|             1|2024-06-24 09:03:13| Tnx_date|
+---+--------------+-----------+--------------------+--------------+-------------------+-------------------+--------------+-------------------+---------+



1

In [6]:
df.show()

+---+--------------+-----------+--------------------+--------------+-------------------+-------------------+--------------+-------------------+---------+
| id|  Schema_names|Table_names|            location|hdfs_file_name|    start_date_time|      end_date_time|is_incremental|     execution_date|inc_field|
+---+--------------+-----------+--------------------+--------------+-------------------+-------------------+--------------+-------------------+---------+
|  1|test_database1|sample_data|hdfs://localhost:...|   sample_data|2023-06-28 00:00:01|2023-06-28 23:59:59|             1|2024-06-24 09:03:13| Tnx_date|
+---+--------------+-----------+--------------------+--------------+-------------------+-------------------+--------------+-------------------+---------+



In [7]:
def sql_updater(index):
    try:
        with connection.cursor() as cursor:
            try:
                exec_date = f'update `main_database`.cf_etl_table set execution_date = current_timestamp where id = {index + 1}'
                cursor.execute(exec_date)
            except Exception as e:
                print(f"Error updating execution date: {e}")
                return

            try:
                start_date = f'update `main_database`.cf_etl_table set start_date_time = date_add(start_date_time, interval 1 day)'
                cursor.execute(start_date)
            except Exception as e:
                print(f"Error updating start date: {e}")
                return

            try:
                end_date = f'update `main_database`.cf_etl_table set end_date_time = date_add(end_date_time, interval 1 day)'
                cursor.execute(end_date)
            except Exception as e:
                print(f"Error updating end date: {e}")
                return

            try:
                connection.commit()
            except Exception as e:
                print(f"Error committing transaction: {e}")
                connection.rollback()
    except Exception as e:
        print(f"Error in SQL updater: {e}")

In [8]:
def etl(url, table_name, properties):
    try:
        df = spark.read.jdbc(url=url, table=table_name, properties=properties)
    except Exception as e:
        print(f"Error reading from JDBC: {e}")
        return
    rows = df.collect()
    for row in rows:
        is_inc = row['is_incremental']
        location,hdfs_file_name,inc_field,database_name,table_name = row['location'],row['hdfs_file_name'],row['inc_field'],row['Schema_names'],row['Table_names']
        if is_inc:
            start_date,end_date= row['start_date_time'],row['end_date_time']
            hdfs_path = f'{location}{hdfs_file_name}'
            jdbc_url = f"jdbc:mysql://localhost:3306/{database_name}"
            query = f"(SELECT * FROM {database_name}.{table_name} WHERE {inc_field} BETWEEN '{start_date}' AND '{end_date}') as selected_data"          
            dataframe = spark.read.jdbc(url=jdbc_url, table=query, properties=properties)
            dataframe.write.mode('append').parquet(hdfs_path)
            sql_updater(rows.index(row))
        else:
            jdbc_url = f"jdbc:mysql://localhost:3306/{database_name}"
            dataframe = spark.read.jdbc(url=jdbc_url, table=table_name, properties=properties)
            hdfs_path = f'{location}{hdfs_file_name}'
            dataframe.write.mode("overwrite").parquet(hdfs_path)  

In [9]:
etl(url,'cf_etl_table',properties)

In [10]:
data = spark.read.parquet('hdfs://localhost:19000/anurag/sample_data')

In [12]:
df = spark.read.jdbc(url=url, table='cf_etl_table', properties=properties)